### Loading necessary libraries

In [7]:

import os
import glob
import time
import librosa
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D, MaxPooling2D
from keras.optimizers import SGD

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from keras.optimizers import SGD


### Extracting features for each file

In [24]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

def parse_audio_files(parent_dir,sub_dirs,file_ext='*.wav'):
    features, labels = np.empty((0,193)), np.empty(0)
    for fn in glob.glob(os.path.join(parent_dir, sub_dirs, file_ext)):
        mfccs, chroma, mel, contrast,tonnetz = extract_feature(fn)
        ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
        features = np.vstack([features,ext_features])
        labels = np.append(labels, fn.split('/')[3].split('-')[3].split('.')[0])
    return np.array(features), np.array(labels, dtype = np.int)

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [25]:
start_time = time.time()

parent_dir = '../ESC-50-master'
sub_dir = '10_sec_audio'

features, labels = parse_audio_files(parent_dir,sub_dir)
labels = one_hot_encode(labels)

print("--> %s seconds <--" % (time.time() - start_time))

/Users/supriodutta/AnacondaInstallation/anaconda3/lib/python3.6/site-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


--> 2101.5237140655518 seconds <--


## Building CNN

/Users/supriodutta/AnacondaInstallation/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/supriodutta/AnacondaInstallation/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [21]:
from sklearn.model_selection import train_test_split

In [22]:
# Prepare the data
X = features
y = labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [24]:
model = Sequential()

model.add(Conv1D(64, 3, activation='relu', input_shape=(193, 1)))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(50, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 191, 64)           256       
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 189, 64)           12352     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 63, 64)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 61, 128)           24704     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 59, 128)           49280     
_________________________________________________________________
global_average_pooling1d_2 ( (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [25]:
start_time = time.time()

X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

model.fit(X_train, y_train, batch_size=50, epochs=1000)
score, acc = model.evaluate(X_test, y_test, batch_size=16)

print('Test score:', score)
print('Test accuracy:', acc)

print("--> %s seconds <--" % (time.time() - start_time))

Epoch 1/1000
1400/1400 [==============================] - 1s 913us/step - loss: 3.9061 - acc: 0.0300
Epoch 2/1000
1400/1400 [==============================] - 1s 648us/step - loss: 3.8006 - acc: 0.0507
Epoch 3/1000
1400/1400 [==============================] - 1s 643us/step - loss: 3.7179 - acc: 0.0514
Epoch 4/1000
1400/1400 [==============================] - 1s 592us/step - loss: 3.6028 - acc: 0.0729
Epoch 5/1000
1400/1400 [==============================] - 1s 626us/step - loss: 3.5124 - acc: 0.0857
Epoch 6/1000
1400/1400 [==============================] - 1s 617us/step - loss: 3.4221 - acc: 0.0886
Epoch 7/1000
1400/1400 [==============================] - 1s 638us/step - loss: 3.3610 - acc: 0.1100
Epoch 8/1000
1400/1400 [==============================] - 1s 618us/step - loss: 3.2680 - acc: 0.1293
Epoch 9/1000
1400/1400 [==============================] - 1s 684us/step - loss: 3.2129 - acc: 0.1471
Epoch 10/1000
1400/1400 [==============================] - 1s 676us/step - loss: 3.1379 - a

1400/1400 [==============================] - 1s 694us/step - loss: 0.5782 - acc: 0.8114
Epoch 161/1000
1400/1400 [==============================] - 1s 632us/step - loss: 0.5506 - acc: 0.8329
Epoch 162/1000
1400/1400 [==============================] - 1s 650us/step - loss: 0.5697 - acc: 0.8200
Epoch 163/1000
1400/1400 [==============================] - 1s 636us/step - loss: 0.5853 - acc: 0.8114
Epoch 164/1000
1400/1400 [==============================] - 1s 668us/step - loss: 0.5226 - acc: 0.8386
Epoch 165/1000
1400/1400 [==============================] - 1s 659us/step - loss: 0.5239 - acc: 0.8271
Epoch 166/1000
1400/1400 [==============================] - 1s 641us/step - loss: 0.5485 - acc: 0.8300
Epoch 167/1000
1400/1400 [==============================] - 1s 656us/step - loss: 0.5297 - acc: 0.8286
Epoch 168/1000
1400/1400 [==============================] - 1s 695us/step - loss: 0.5045 - acc: 0.8514
Epoch 169/1000
1400/1400 [==============================] - 1s 700us/step - loss: 0.5449

1400/1400 [==============================] - 1s 714us/step - loss: 0.2031 - acc: 0.9300
Epoch 319/1000
1400/1400 [==============================] - 1s 661us/step - loss: 0.2216 - acc: 0.9336
Epoch 320/1000
1400/1400 [==============================] - 1s 695us/step - loss: 0.1992 - acc: 0.9336
Epoch 321/1000
1400/1400 [==============================] - 1s 675us/step - loss: 0.2328 - acc: 0.9279
Epoch 322/1000
1400/1400 [==============================] - 1s 668us/step - loss: 0.2164 - acc: 0.9364
Epoch 323/1000
1400/1400 [==============================] - 1s 687us/step - loss: 0.1839 - acc: 0.9407
Epoch 324/1000
1400/1400 [==============================] - 1s 679us/step - loss: 0.2182 - acc: 0.9343
Epoch 325/1000
1400/1400 [==============================] - 1s 678us/step - loss: 0.2003 - acc: 0.9357
Epoch 326/1000
1400/1400 [==============================] - 1s 674us/step - loss: 0.2290 - acc: 0.9250
Epoch 327/1000
1400/1400 [==============================] - 1s 668us/step - loss: 0.2006

1400/1400 [==============================] - 1s 660us/step - loss: 0.1364 - acc: 0.9543
Epoch 477/1000
1400/1400 [==============================] - 1s 685us/step - loss: 0.1689 - acc: 0.9571
Epoch 478/1000
1400/1400 [==============================] - 1s 677us/step - loss: 0.1436 - acc: 0.9579
Epoch 479/1000
1400/1400 [==============================] - 1s 656us/step - loss: 0.1507 - acc: 0.9657
Epoch 480/1000
1400/1400 [==============================] - 1s 679us/step - loss: 0.1552 - acc: 0.9493
Epoch 481/1000
1400/1400 [==============================] - 1s 711us/step - loss: 0.1294 - acc: 0.9550
Epoch 482/1000
1400/1400 [==============================] - 1s 670us/step - loss: 0.1974 - acc: 0.9521
Epoch 483/1000
1400/1400 [==============================] - 1s 651us/step - loss: 0.1322 - acc: 0.9586
Epoch 484/1000
1400/1400 [==============================] - 1s 629us/step - loss: 0.1253 - acc: 0.9629
Epoch 485/1000
1400/1400 [==============================] - 1s 691us/step - loss: 0.1686

1400/1400 [==============================] - 1s 649us/step - loss: 0.1581 - acc: 0.9579
Epoch 635/1000
1400/1400 [==============================] - 1s 691us/step - loss: 0.1466 - acc: 0.9600
Epoch 636/1000
1400/1400 [==============================] - 1s 672us/step - loss: 0.1236 - acc: 0.9643
Epoch 637/1000
1400/1400 [==============================] - 1s 681us/step - loss: 0.1654 - acc: 0.9571
Epoch 638/1000
1400/1400 [==============================] - 1s 677us/step - loss: 0.1047 - acc: 0.9714
Epoch 639/1000
1400/1400 [==============================] - 1s 692us/step - loss: 0.1449 - acc: 0.9571
Epoch 640/1000
1400/1400 [==============================] - 1s 676us/step - loss: 0.1159 - acc: 0.9700
Epoch 641/1000
1400/1400 [==============================] - 1s 676us/step - loss: 0.1758 - acc: 0.9557
Epoch 642/1000
1400/1400 [==============================] - 1s 703us/step - loss: 0.0914 - acc: 0.9679
Epoch 643/1000
1400/1400 [==============================] - 1s 668us/step - loss: 0.1001

1400/1400 [==============================] - 1s 669us/step - loss: 0.1304 - acc: 0.9650
Epoch 793/1000
1400/1400 [==============================] - 1s 698us/step - loss: 0.1090 - acc: 0.9700
Epoch 794/1000
1400/1400 [==============================] - 1s 688us/step - loss: 0.1107 - acc: 0.9743
Epoch 795/1000
1400/1400 [==============================] - 1s 672us/step - loss: 0.2106 - acc: 0.9536
Epoch 796/1000
1400/1400 [==============================] - 1s 670us/step - loss: 0.1251 - acc: 0.9629
Epoch 797/1000
1400/1400 [==============================] - 1s 681us/step - loss: 0.1117 - acc: 0.9729
Epoch 798/1000
1400/1400 [==============================] - 1s 683us/step - loss: 0.1219 - acc: 0.9700
Epoch 799/1000
1400/1400 [==============================] - 1s 659us/step - loss: 0.1287 - acc: 0.9600
Epoch 800/1000
1400/1400 [==============================] - 1s 651us/step - loss: 0.1238 - acc: 0.9643
Epoch 801/1000
1400/1400 [==============================] - 1s 646us/step - loss: 0.1615

1400/1400 [==============================] - 1s 679us/step - loss: 0.1047 - acc: 0.9707
Epoch 951/1000
1400/1400 [==============================] - 1s 691us/step - loss: 0.1307 - acc: 0.9671
Epoch 952/1000
1400/1400 [==============================] - 1s 668us/step - loss: 0.0998 - acc: 0.9650
Epoch 953/1000
1400/1400 [==============================] - 1s 661us/step - loss: 0.1220 - acc: 0.9657
Epoch 954/1000
1400/1400 [==============================] - 1s 676us/step - loss: 0.1065 - acc: 0.9721
Epoch 955/1000
1400/1400 [==============================] - 1s 660us/step - loss: 0.1072 - acc: 0.9664
Epoch 956/1000
1400/1400 [==============================] - 1s 683us/step - loss: 0.1250 - acc: 0.9664
Epoch 957/1000
1400/1400 [==============================] - 1s 643us/step - loss: 0.0859 - acc: 0.9757
Epoch 958/1000
1400/1400 [==============================] - 1s 654us/step - loss: 0.1416 - acc: 0.9679
Epoch 959/1000
1400/1400 [==============================] - 1s 661us/step - loss: 0.1084

### Predicting the class of a sound taken from basin tap

In [5]:
test = "/Users/supriodutta/MyFiles/SuprioFiles/JobHunt/internship/mBreath/FinalRound/sound.wav"

In [6]:
mfccs, chroma, mel, contrast,tonnetz = extract_feature(test)
ext_features_test = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
features_test = np.vstack([ext_features_test])
features_test = np.expand_dims(features_test, axis=2)

NameError: name 'extract_feature' is not defined

In [71]:
predicted_classes = model.predict(features_test)    # Predicting the class
np.argmax(np.round(predicted_classes),axis=1)
esc50 = pd.read_csv("ESC-50-master/meta/esc50.csv") # Loading ESC-50 to get the class name
list(esc50.category[ecs50.target == 18][:1])        # Getting the class name

['toilet_flush']

In [4]:
esc50.category

NameError: name 'esc50' is not defined